# 支持向量机 Support Vector Machines

本节包括以下内容：
1. 间隔 Margin
2. 记号 Notation
3. 函数间隔和几何间隔 Functional and geometric margins
4. 最大间隔分类器 The optimal margin classifier
5. 拉格朗日对偶性 Lagrange duality
6. 最大间隔分类器 Optimal margin classifier
7. 核方法 Kernels
8. 正则化、不可分的情况 Regularization and the non-separable case
9. SMO算法 The SMO algorithm
10. SMO


### 1. 间隔 Margin

回忆线性回归，概率 $p(y=1|x; \theta)$ 由 $h_\theta(x) = g(\theta^Tx)$ 计算而得。如果 $h_\theta(x) \geq 0.5$，则我们预测输入 $x$ 为正类，换言之，当且仅当 $\theta^Tx \geq 0$ 时，模型预测 $x$ 为正类。并且，根据我们的概率假设，$\theta^Tx$ 越大，$x$ 取正类的概率就越大，或者说，我们对于预测 $x$ 为正类的信心就越大。

也就是说，如果 $\theta^Tx \gg 0$，那么我们可以非常自信地预测 $y=1$。类似的，如果 $\theta^Tx \ll 0$，则我们可以非常自信地预测 $y=0$。直觉上看，如果我们的成本函数考虑这一点，那么我们可以得到一个不错的分类器。对此，稍后我们将会引入函数间隔的概念。

换一种思路来看，$\theta^Tx=0$ 代表决策边界（也称为**分割超平面 separating hyperplane**），离决策边界越远的点，我们对准确预测其分类的信心越大。直觉上看，在数据集线性可分的前提下，在所有决策边界集合中，使得样本点尽可能远离的决策边界，通常代表着更好的模型。对此，稍后我们会引入几何间隔的概念。

### 2. 记号 Notation

对于支持向量机，我们将引入和广义线性模型所不同的符号来阐述二元分类问题。给定标签 $y$ 和特征 $x$，令 $y \in \{-1, 1\}$（而不是 $\{0, 1\}$），同时我们不再使用 $\theta$ 作为参数向量，而是使用 $w, b$ 来描述分类器
$$ h_{w, b}(x) = g(w^Tx + b) $$

这里，当 $z \geq 0$ 时，$g(z)=1$，否则 $g(z) = -1$。$w, b$ 的记号，使我们将截距项 $b$ 从其它参数独立出来，从而我们也不需要再假定 $x_0=1$，因而这里 $x$ 是 $n$ 维向量，而不是 $n+1$ 维向量。也就是说，$b$ 就是之前的 $\theta_0$，而 $w$ 是之前的 $[\theta_1...\theta_n]^T$

注意到，根据我们对 $g$ 的上述定义，我们的分类器将直接预测 $1$ 或 $-1$（类比感知器），而不再需要和逻辑回归一样，包含预测 $y=1$ 的概率这个中间步骤。

### 3. 函数间隔和几何间隔 Functional and Geometric Margins

给定一个训练样本 $(x^{(i)}, y^{(i)})$，定义该训练样本 $(w, b)$ 的**函数间隔 functional margin**为
$$ \hat{\gamma}^{(i)} = y^{(i)}(w^Tx+b) $$

注意到，如果 $y^{(i)}=1$，要使函数间隔更大（也即使我们的预测更有信心更准确），则需要 $w^Tx+b$ 是一个非常大的正数。相反，如果 $^{(i)}=-1$，要使函数间隔更大，则需要 $w^Tx+b$ 是一个（绝对值）非常大的负数。另外，当 $y^{(i)}(w^Tx+b) > 0$ 时，我们对这个训练样本的预测是准确的。因此，一个非常大的函数间隔，同时代表着置信度高以及正确的分类。

对于上述给定的线性分类器 $g$ （取值范围为 $\{-1, 1\}$），函数间隔的一个特性使其并不能成为置信度的很好测量指标。考虑如下情况：将 $w$ 替换为 $2w$，$b$ 替换为 $2b$，由于 $g(w^Tx+b)=g(2w^Tx+2b)$，分类结果 $h_{w,b}(x)$ 将保持不变。然而，这个时候函数间隔扩大了两倍。换言之，我们可以同比例地扩大参数 $w, b$，从而以任意倍数扩大函数间隔，但这个操作对分类器本身的效果而言，不带来任何变化。从直觉上看，我们应该对参数做归一化操作，使得 $||w||_2=1$，替换 $(w,b)$ 为 $(\frac{w}{||w||_2}, \frac{b}{||w||_2})$，在此基础上，再计算函数间隔。之后我们将会回到这个问题。

给定一个训练集，我们定义训练集 $S$ 的函数间隔为所有单个训练样本的函数间隔的最小值，即
$$ \hat{\gamma} = \min_{i=1,...,m}\hat{\gamma}^{(i)} $$

接下来，考虑**几何间隔 geometric margin**

注意到，向量 $w$ 和分割超平面总是正交的，考虑处于正类的点 $A$，其特征为 $x^{(i)}$，标签为 $y^{(i)}=1$，假设点 $A$ 到决策边界的距离为 $\gamma^{(i)}$，相交点为 $B$。

这里 $\frac{w}{||w||}$ 是 $w$ 方向的单位向量，点 $A$ 可以用向量 $x^{(i)}$ 表示。因此点 $B$ 就是 $x^{(i)} - \gamma^{(i)} \cdot \frac{w}{||w||}$，由于点 $B$ 处理决策边界上，满足 $w^Tx+b=0$，因此
$$ w^T(x^{(i)}-\gamma^{(i)}\frac{w}{||w||}+b=0) $$
解方程得
$$ \gamma^{(i)} = \frac{w^Tx^{(i)}+b}{||w||} = (\frac{w}{||w||})^Tx^{(i)} + \frac{b}{||w||} $$

这是正类的情况，对于正类和反类的情况
$$ \gamma^{(i)} = y^{(i)}(\frac{w^Tx^{(i)}+b}{||w||} = (\frac{w}{||w||})^Tx^{(i)} + \frac{b}{||w||}) $$

注意到，当 $||w||=1$ 时，函数间隔等于几何间隔。并且，几何间隔不会收到参数扩大缩小的影响。也即是说，我们可以按任意约束对 $w, b$ 进行缩放，而不改变函数间隔。

最后，给定一个训练集，我们同样定义训练集 $S$ 的几何间隔为所有单个训练样本的几何间隔的最小值，即
$$ \hat{\gamma} = \min_{i=1,...,m}\hat{\gamma}^{(i)} $$